In [3]:
from tdc.multi_pred import DrugRes
import numpy as np
import matplotlib.pyplot as plot
import torch
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm


In [65]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
    def forward(self, x):
        return self.linear(x)
    

def train_loop(dataloader, model, loss_fn, optimizer):
    train_loss = 0
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X.float())
        loss = loss_fn(pred, y.float())
        train_loss += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return train_loss / len(dataloader)

def test_loop(dataloader, model, loss_fn):
    test_loss = 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X.float())
            test_loss += loss_fn(pred, y.float()).item()
    return test_loss / len(dataloader)
    

In [114]:
class Drug():
    def __init__(self, drug_id, drug_smiles):
        self.id = drug_id
        self.smiles = drug_smiles
#         self.train_dataloader = train_dataloader
#         self.test_dataloader = test_dataloader
    
class Model():
    def __init__(self):
        data = DrugRes(name = 'GDSC1')
        split = data.get_split()
        self.train = split['train']
        self.test = split['test']
        self.valid = split['valid']
        self.drug_IDs = self.train['Drug_ID'].unique()
        assert(set(self.test['Drug_ID'].unique().flatten()) == \
               set(self.drug_IDs.flatten())) # make sure train, test set involves same set of drugs
        
        self.drugs = {}
        self.submodels = {}
        self.batch_size = 256
            
        
    def train_all_submodels(self):
        epochs = 1e5
        lr = 1e-7
        loss_fn = torch.nn.MSELoss()
        print("Initializing and training submodel for each drug")
        for ID in tqdm(self.drug_IDs):
            train_X = torch.from_numpy(np.vstack(self.train[self.train['Drug_ID']==ID]['Cell Line'].to_numpy()))
            train_Y = torch.from_numpy(np.vstack(self.train[self.train['Drug_ID']==ID]['Y'].to_numpy()))
            test_X = torch.from_numpy(np.vstack(self.test[self.test['Drug_ID']==ID]['Cell Line'].to_numpy()))
            test_Y = torch.from_numpy(np.vstack(self.test[self.test['Drug_ID']==ID]['Y'].to_numpy()))
            self.drugs[ID] = Drug(ID, None)  
            self.submodels[ID] = LogisticRegression(train_X.shape[1], 1).to('cpu')
            optimizer = torch.optim.Adam(self.submodels[ID].parameters(), lr=lr)
            train_dataloader = DataLoader(TensorDataset(train_X, train_Y),\
                                               self.batch_size, shuffle = True)
            test_dataloader = DataLoader(TensorDataset(test_X, test_Y),\
                                              self.batch_size, shuffle = True)
            train_loss, test_loss = [100], [100]
            counter = 0
            for t in range(epochs):
                train_l = train_loop(train_dataloader, self.submodels[ID], loss_fn, optimizer)
                test_l = test_loop(test_dataloader, self.submodels[ID], loss_fn)
                if (t + 1) % 100 == 0:
                    print(f"Epoch {t+1}\n-------------------------------")
                    train_loss.append(train_l)
                    test_loss.append(test_l)
                    print(f"Avg train loss: {train_loss[-1]:.8f}")
                    print(f"Avg test loss: {test_loss[-1]:.8f}")
                    if test_loss[-1] >= test_loss[-2]: # simple early stopping
                        counter+=1
                    else:
                        counter=max(0, counter-1)
                    if counter>=3:
                        break
    def evaluate(criterion = 'MSE'):
        if criterion == 'MAE':
            loss_fn = torch.nn.L1Loss()
        elif criterion == 'MSE':
            loss_fn = torch.nn.MSELoss()
        
        total_loss = 0
        valid_X = self.valid['Cell Line'].to_numpy()
        valid_Y = self.valid['Y'].to_numpy()
        valid_IDs = self.valid['Drug_ID'].to_numpy()
        for X, y, ID in zip(valid_X, valid_Y, valid_IDs):
            pred = self.submodels[ID](torch.from_numpy(X).float)
            total_loss += loss_fn(pred, torch.from_numpy(y).float()).item()
        return total_loss / len(valid_X)

In [115]:
model = Model()
model.train_all_submodels()

Found local copy...
Loading...
Done!


Initializing and training submodel for each drug


  0%|                                                   | 0/208 [00:00<?, ?it/s]

Epoch 100
-------------------------------
Avg train loss: 47.99098587
Avg test loss: 48.97399139
Epoch 200
-------------------------------
Avg train loss: 37.81008911
Avg test loss: 38.66719055
Epoch 300
-------------------------------
Avg train loss: 29.41374016
Avg test loss: 30.15335083
Epoch 400
-------------------------------
Avg train loss: 22.58801079
Avg test loss: 23.21865654
Epoch 500
-------------------------------
Avg train loss: 17.12712097
Avg test loss: 17.65731049
Epoch 600
-------------------------------
Avg train loss: 12.83506203
Avg test loss: 13.27335358
Epoch 700
-------------------------------
Avg train loss: 9.52765560
Avg test loss: 9.88251877
Epoch 800
-------------------------------
Avg train loss: 7.03369856
Avg test loss: 7.31355238
Epoch 900
-------------------------------
Avg train loss: 5.19754505
Avg test loss: 5.41065931
Epoch 1000
-------------------------------
Avg train loss: 3.88040400
Avg test loss: 4.03485203
Epoch 1100
--------------------------

Epoch 8700
-------------------------------
Avg train loss: 1.17434549
Avg test loss: 1.11092901
Epoch 8800
-------------------------------
Avg train loss: 1.16973281
Avg test loss: 1.10829997
Epoch 8900
-------------------------------
Avg train loss: 1.16505933
Avg test loss: 1.10569155
Epoch 9000
-------------------------------
Avg train loss: 1.16032660
Avg test loss: 1.10310709
Epoch 9100
-------------------------------
Avg train loss: 1.15553391
Avg test loss: 1.10054755
Epoch 9200
-------------------------------
Avg train loss: 1.15068400
Avg test loss: 1.09801888
Epoch 9300
-------------------------------
Avg train loss: 1.14577651
Avg test loss: 1.09552157
Epoch 9400
-------------------------------
Avg train loss: 1.14081025
Avg test loss: 1.09305739
Epoch 9500
-------------------------------
Avg train loss: 1.13578749
Avg test loss: 1.09062958
Epoch 9600
-------------------------------
Avg train loss: 1.13070655
Avg test loss: 1.08823943
Epoch 9700
-----------------------------

...

Epoch 8600
-------------------------------
Avg train loss: 0.66991669
Avg test loss: 1.06387115
Epoch 8700
-------------------------------
Avg train loss: 0.64651865
Avg test loss: 1.06246257
Epoch 8800
-------------------------------
Avg train loss: 0.68924314
Avg test loss: 1.06234038
Epoch 8900
-------------------------------
Avg train loss: 0.64408970
Avg test loss: 1.06177127
Epoch 9000
-------------------------------
Avg train loss: 0.65559775
Avg test loss: 1.06235373
Epoch 9100
-------------------------------
Avg train loss: 0.66291171
Avg test loss: 1.06115055
Epoch 9200
-------------------------------
Avg train loss: 0.64032650
Avg test loss: 1.06061113
Epoch 9300
-------------------------------
Avg train loss: 0.65313077
Avg test loss: 1.06015277
Epoch 9400
-------------------------------
Avg train loss: 0.66124374
Avg test loss: 1.05959797
Epoch 9500
-------------------------------
Avg train loss: 0.64049751
Avg test loss: 1.05966866
Epoch 9600
-----------------------------

 46%|████████████████▍                   | 95/208 [8:20:31<14:44:32, 469.67s/it]

Epoch 10000
-------------------------------
Avg train loss: 0.66415840
Avg test loss: 1.05747247
Epoch 100
-------------------------------
Avg train loss: 0.97154313
Avg test loss: 1.23509049
Epoch 200
-------------------------------
Avg train loss: 0.96862990
Avg test loss: 1.22326338
Epoch 300
-------------------------------
Avg train loss: 0.95969677
Avg test loss: 1.20677912
Epoch 400
-------------------------------
Avg train loss: 0.98624486
Avg test loss: 1.18897879
Epoch 500
-------------------------------
Avg train loss: 0.96447062
Avg test loss: 1.16864085
Epoch 600
-------------------------------
Avg train loss: 0.97767729
Avg test loss: 1.14726949
Epoch 700
-------------------------------
Avg train loss: 0.96585321
Avg test loss: 1.12531602
Epoch 800
-------------------------------
Avg train loss: 0.95058745
Avg test loss: 1.10370350
Epoch 900
-------------------------------
Avg train loss: 0.87864155
Avg test loss: 1.08240306
Epoch 1000
-------------------------------
Avg t

 46%|████████████████▍                   | 95/208 [8:25:42<10:01:31, 319.39s/it]


KeyboardInterrupt: 

In [139]:
loss_fn = torch.nn.MSELoss()
valid_X = model.test['Cell Line'].to_numpy()
valid_Y = model.test['Y'].to_numpy()
valid_IDs = model.test['Drug_ID'].to_numpy()
counter = 0
total_loss = 0
for X, y, ID in zip(valid_X, valid_Y, valid_IDs):
    if ID in model.submodels:
        counter += 1
        pred = model.submodels[ID](torch.from_numpy(X).float())
        total_loss += loss_fn(pred, torch.from_numpy(np.asarray(y)).float()).item()

avg_loss = total_loss / counter
print(avg_loss)





1.902758153684913
